#read the readme to know exactly what code is doing. 

Here is the breakdown of how I want to build the code. 

- We will use requests to load the first page of the results. 
- Store the links to each job description. I don't want to use the in page feature of the job description. That makes the HTML too complicated.
- Once we have the links, use requests to get content for each one. 
- Use beutiful soup to scrape the job description. 
- The idea is to give the user the choice of skills he wants the summary. For the initial process I will provide a list of skills and check the results to see if everything works clearly. Then move on to user inputs. 
- Once the code works for let's say 10 or 20 job searches, we can scale it to incorporate more companies.
- Be ethical. I will keep a cap on how many companies we can search in the end. Let's not be scummy and inundate the indeed server.


In [1]:
#let's begin 

import pyforest 
import requests 
from bs4 import BeautifulSoup

In [2]:
job_page = requests.get('https://www.indeed.com/jobs?q=data+scientist&l=United+States')
job_page

<Response [200]>

In [3]:
first_page_content = job_page.text

In [4]:
parser = BeautifulSoup(first_page_content, 'html.parser')

In [76]:
test = parser.body.find('div', attrs = {'data-tn-component':'organicJob'})

In [83]:
test.a.text

'\nData Scientist, Amazon Studios'

In [84]:
job_title = test.a.text.strip()

In [85]:
job_title

'Data Scientist, Amazon Studios'

In [86]:
job_link = test.a.get('href')

In [87]:
job_link

'/rc/clk?jk=abb4a8359abeeb2e&fccid=fe2d21eef233e94a&vjs=3'

So we pretty much figured out how to get the job name and link. 